### 🔹 Voting Regression  

Voting Regression is an **ensemble learning method** that combines the predictions of multiple different regression models to produce a more robust and accurate final prediction.  
Unlike bagging or boosting, which use homogeneous or sequential models, voting combines **heterogeneous models** (e.g., Linear Regression, Decision Tree, Random Forest, etc.) in a **parallel** fashion.  

The main idea:  
- Train several diverse base regressors on the same dataset.  
- Combine their predictions using a **voting strategy**:  
  - **Hard voting** (for classification) — majority rule.  
  - **Soft voting / averaging** (for regression) — average of all predictions.  
- This reduces the risk of relying on a single model’s bias or variance.  

Mathematically:  

$$
\hat{Y} = \frac{1}{N} \sum_{i=1}^{N} f_i(X)
$$  

where:  
- $(f_i(X))$ is the prediction from the \(i\)-th model,  
- $(N)$ is the total number of base models,  
- $(\hat{Y})$ is the final averaged prediction.  

Voting Regression helps us to:  
- Combine the **strengths of multiple algorithms**.  
- **Reduce variance and bias** simultaneously.  
- Achieve **more stable and generalizable** performance across datasets.  

In this notebook, we will implement **Voting Regression** using a mix of models (e.g., Linear Regression, Decision Tree, and Random Forest) and compare its performance with individual models 🚀.  


# --------------------------------------------------------------------------

# import dataset

In [1]:
# from google.colab import files, drive

# up = files.upload()
# drive.mount('/content/drive')

In [2]:
import pandas as pd

df = pd.read_csv('dataset.csv')
df.head()

,f1,f2,f3,T
0,2.0,4,8.5,196
1,2.4,4,9.6,221
2,1.5,4,5.9,136
3,3.5,6,11.1,255
4,3.5,6,10.6,244


# cleaning

In [3]:
# clean the data

# encoding

In [4]:
# encode the data

# define x , y

In [5]:
import numpy as np

x = df[['f1', 'f2', 'f3']].values
y = df['T'].values

# spliting

In [6]:
# # finding best random state 

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
# base_estimators = [
#     ('lr', LinearRegression()),
#     ('rf', RandomForestRegressor(random_state=1)),
#     ('svr', SVR())
# ]
# from sklearn.ensemble import VotingRegressor
# vr = VotingRegressor(estimators=base_estimators)
# from sklearn.metrics import r2_score

# import time
# t1 = time.time()



# lst = []
# for i in range(1, 10):
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=i)
#     x_train = mms.fit_transform(x_train)
#     x_test = mms.transform(x_test)
    
#     vr.fit(x_train, y_train)
    
#     yhat_test = vr.predict(x_test)
#     r2 = r2_score(y_test, yhat_test)
#     lst.append(r2)

# t2 = time.time()
# print(f"run time: {round((t2 - t1) / 60, 2)} min")
# print(f"r2_score: {round(max(lst), 4)}")
# print(f"random_state: {np.argmax(lst) + 1}")

run time: 0.05 min
r2_score: 0.9332
random_state: 4


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# scaling

In [8]:
# scaling depends on the base models used in the stacking ensemble.
# If any base model requires feature scaling, then the data should be scaled accordingly.

In [9]:
# from sklearn.preprocessing import MinMaxScaler

# mms = MinMaxScaler().fit(x_train)
# x_train = mms.transform(x_train)
# x_test = mms.transform(x_test)

# Define base learners

In [10]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

In [11]:
base_estimators = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('xgbr', XGBRegressor()),
    ('svr', SVR())
]

# Initialize the Voting Regressor

In [12]:
vr = VotingRegressor(estimators=base_estimators)

# fit the model

In [14]:
# estimators, weights=None, n_jobs=None, verbose=False

In [15]:
vr.fit(x_train, y_train)

VotingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                            ('xgbr',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None, ...)),
                            ('svr', SVR())])

# predict test data

In [16]:
yhat_test = vr.predict(x_test)

# evaluate the model

In [17]:
from sklearn.metrics import r2_score

print("r2-score (train data): %0.4f" % r2_score(y_train, vr.predict(x_train)))
print("r2-score (test data): %0.4f" % r2_score(y_test, yhat_test))

r2-score (train data): 0.9726
r2-score (test data): 0.9437


In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(f"MSE  (train data): {mean_squared_error(y_train, vr.predict(x_train))}")
print(f"RMSE (train data): {np.sqrt(mean_squared_error(y_train, vr.predict(x_train)))}")
print(f"MAE  (train data): {mean_absolute_error(y_train, vr.predict(x_train))}")
print('------------')
print(f"MSE  (test data): {mean_squared_error(y_test, yhat_test)}")
print(f"RMSE (test data): {np.sqrt(mean_squared_error(y_test, yhat_test))}")
print(f"MAE  (test data): {mean_absolute_error(y_test, yhat_test)}")

MSE  (train data): 108.33012312611862
RMSE (train data): 10.408175782821822
MAE  (train data): 5.855610418387305
------------
MSE  (test data): 236.04192284754924
RMSE (test data): 15.363655907613566
MAE  (test data): 7.020889179055189


# predict new data

In [19]:
# new_data = mms.transform([[2, 4, 8.5]])
# vr.predict(new_data)

vr.predict([[2, 4, 8.5]])

array([197.31448548])

# save the model

In [20]:
# import joblib
# joblib.dump(vr, 'vr_model.pkl')

# load the model

In [21]:
# import joblib
# vr = joblib.load('vr_model.pkl')